# [SIIM-FISABIO-RSNA COVID-19 Detection](https://www.kaggle.com/c/siim-covid19-detection)
> Identify and localize COVID-19 abnormalities on chest radiographs

![](https://storage.googleapis.com/kaggle-competitions/kaggle/26680/logos/header.png)

# Reference
Check this amazing notebook, [How To Create TFRecords](https://www.kaggle.com/cdeotte/how-to-create-tfrecords) by [Chris Deotte](https://www.kaggle.com/cdeotte)

# How to Create TFRecord

In [ ]:
SEED  = 42
FOLDS = 5
DIM   = 512

# Importing Packages

In [ ]:
import numpy as np 
import pandas as pd 
import os, shutil
from glob import glob
from sklearn.cluster import KMeans
from tqdm.notebook import tqdm
from sklearn.preprocessing import LabelEncoder
import random
tqdm.pandas()

In [ ]:
np.random.seed(SEED)
random.seed(SEED)

In [ ]:
train_df = pd.read_csv(f'../input/siimcovid19-{DIM}-jpg-image-dataset/train.csv')
train_df['image_path'] = F'../input/siimcovid19-{DIM}-jpg-image-dataset/train/'+train_df.image_id+'.jpg'
train_df.head(2)

# ClassName and ClassLabel Map

In [ ]:
name2label = {'Typical Appearance': 3,
 'Indeterminate Appearance': 1,
 'Atypical Appearance': 2,
 'Negative for Pneumonia': 0}
class_names = list(name2label.keys())
label2name = {v:k for k, v in name2label.items()}
train_df['class_name']  = train_df.progress_apply(lambda row:row[class_names].iloc[[row[class_names].values.argmax()]].index.tolist()[0], axis=1)
train_df['class_label'] = train_df.class_name.map(name2label)
train_df.head()

# Stratified KFold by Groups

In [ ]:
from sklearn.model_selection import GroupKFold, StratifiedKFold
gkf  = GroupKFold(n_splits = 5)
train_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(gkf.split(train_df, groups = train_df.StudyInstanceUID.tolist())):
    train_df.loc[val_idx, 'fold'] = fold
train_df.head()

<!-- # Check the signals
From the [dataset information](https://www.kaggle.com/c/seti-breakthrough-listen/overview/data-information),
>>
Not all of the “needle” signals look like diagonal lines, and they may not be present for the entirety of all three “A” observations, but what they do have in common is that they are only present in some or all of the “A” observations (panels **1**, **3**, and **5** in the cadence snippets).

 -->

In [ ]:
import matplotlib.pyplot as plt
import cv2
def load_image(path, dim=DIM, ch=3):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE if ch==None else cv2.IMREAD_COLOR)
    if img.shape[:2]!=(dim,dim) and dim!=-1:
        img = cv2.resize(img, dsize=(dim,dim), interpolation=cv2.INTER_AREA)
    return img

plt.imshow(load_image(train_df.image_path.iloc[100], dim=-1))

# Visualize Channels

# TFRecord Data

In [ ]:
import tensorflow as tf

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# Writng TFRecord (Train)

In [ ]:
def train_serialize_example(feature0, feature1, feature2, feature3):
    feature = {
      'image'         : _bytes_feature(feature0),
      'image_id'      : _bytes_feature(feature1),
      'group'         : _bytes_feature(feature2),    
      'target'        : _int64_feature(feature3),
  }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [ ]:
show=True
folds = train_df.fold.unique().tolist()
for fold in tqdm(folds): # create tfrecord for each fold
    fold_df = train_df[train_df.fold==fold]
    if show:
        print(); print('Writing TFRecord of fold %i :'%(fold))  
    with tf.io.TFRecordWriter('train%.2i-%i.tfrec'%(fold,fold_df.shape[0])) as writer:
        samples = fold_df.shape[0]
#         samples = 200
        it = tqdm(range(samples)) if show else range(samples)
        for k in it: # images in fold
            row = fold_df.iloc[k,:]
            image      = load_image(row['image_path'], dim=DIM)
            image_id   = row['image_id']
            group      = row['StudyInstanceUID']
            target     = np.array(row['class_label'], dtype=np.uint8)
            example  = train_serialize_example(
                cv2.imencode('.jpg', image, (cv2.IMWRITE_JPEG_QUALITY, 96))[1].tobytes(),
                str.encode(image_id),
                str.encode(group),
                target,
                )
            writer.write(example)
        if show:
            filepath = 'train%.2i-%i.tfrec'%(fold,fold_df.shape[0])
            filename = filepath.split('/')[-1]
            filesize = os.path.getsize(filepath)/10**6
            print(filename,':',np.around(filesize, 2),'MB')

# Writing TFRecord (Test)

In [ ]:
# def test_serialize_example(feature0, feature1, feature2):
#     feature = {
#       'image'         : _bytes_feature(feature0),
#       'image_id'      : _bytes_feature(feature1),
#       'group'         : _bytes_feature(feature2),    
#   }
#     example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
#     return example_proto.SerializeToString()

In [ ]:
# show  = True
# folds = 10
# l     = int(np.ceil(test_df.shape[0]/folds))
# for fold in tqdm(range(folds)): # create tfrecord for each fold
#     fold_df = test_df.iloc[l*fold:l*(fold+1)]
#     if show:
#         print(); print('Writing TFRecord of fold %i :'%(fold))  
#     with tf.io.TFRecordWriter('test%.2i-%i.tfrec'%(fold,fold_df.shape[0])) as writer:
#         samples = fold_df.shape[0]
# #         samples = 200
#         it = tqdm(range(samples)) if show else range(samples)
#         for k in it: # images in fold
#             row = fold_df.iloc[k,:]
#             image      = load_signal(row['filepath'], dim=DIM)
#             image      = image[...,::-1] # rgb -> bgr, we'll get the rgb form after decoding the tfrec
#             image_id   = row['id']
#             group      = row['group']
#             example  = test_serialize_example(
#                 cv2.imencode('.png', image)[1].tobytes(),
#                 str.encode(image_id),
#                 str.encode(group),
#                 )
#             writer.write(example)
#         if show:
#             filepath = 'test%.2i-%i.tfrec'%(fold,fold_df.shape[0])
#             filename = filepath.split('/')[-1]
#             filesize = os.path.getsize(filepath)/10**6
#             print(filename,':',np.around(filesize, 2),'MB')

# Reading TFRecord

In [ ]:
import re, math
def decode_image(image_data):
    image = tf.image.decode_png(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image
def prepare_target(target):    
    target = tf.cast(target, tf.float32)            
    target = tf.reshape(target, [1])         
    return target

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image" : tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "target": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    image  = tf.reshape(image, [DIM, DIM, 3])
    target = prepare_target(example['target'])
    return image, target # returns a dataset of (image, label) pairs

def load_dataset(fileids, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(fileids, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(20, seed=SEED)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def count_data_items(fileids):
    # the number of data items is written in the id of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(fileid).group(1)) for fileid in fileids]
    return np.sum(n)

# Visual

In [ ]:
def display_batch(batch, size=2):
    imgs, tars = batch
    plt.figure(figsize=(size*5, 5))
    for img_idx in range(size):
        plt.subplot(1, size, img_idx+1)
        plt.title(f'class: {label2name[tars[img_idx].numpy()[0]]}', fontsize=15)
        plt.imshow(imgs[img_idx,:, :, :])
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout()
    plt.show() 

# Total Images

In [ ]:
# INITIALIZE VARIABLES
IMAGE_SIZE= [DIM,DIM];
BATCH_SIZE = 32
AUTO = tf.data.experimental.AUTOTUNE
TRAINING_FILENAMES = tf.io.gfile.glob('train*.tfrec')
TEST_FILENAMES     = tf.io.gfile.glob('test*.tfrec')
print('There are %i train & %i test images'%(count_data_items(TRAINING_FILENAMES), count_data_items(TEST_FILENAMES)))

# Once Batch Image

In [ ]:
# DISPLAY TRAIN IMAGES
training_dataset = get_training_dataset()
training_dataset = training_dataset.unbatch().batch(20)
train_batch = next(iter(training_dataset))
display_batch(train_batch, 5);

In [ ]:
img, label = train_batch
np.unique(label.numpy(), return_counts=True)